# **🪴 Análasis supervisado de deforestación - Departamento del chocó**

Proyecto educativo de inteligencia artificial

**📌 Enfoque**: Aprendizaje supervisado con Randow Forest

Este notebook ilustra cómo aplicar un modelo supervisado para predecir si un área presenta alta o baja deforestación, utilizando datos geográficos y temporales del Chocó

- **🔗 Fuente de datos**: [AREAS DEFORESTADAS CHOCO](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/AREAS-DEFORESTADAS-CHOCO/iczg-dyt3/about_data)

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Cargar datos
df = pd.read_csv('Datos/AREAS_DEFORESTADAS_CHOCO_20251112.csv')
print('Dimensiones del dataset:', df.shape)
df.head()

Dimensiones del dataset: (7937, 10)


,ID,TIPO GEOMETRIA,AÑO,IMAGEN,CAUSA,AREA_Ha,OBSERVACION,LATITUD,LONGITUD,MUNICIPIO
0,7463,Polygon,"2,020",T18PTQ_20200201T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.126512,Ninguna,"8° 22' 43,726"" N","77° 14' 19,019"" W",ACANDÍ
1,4659,Polygon,"2,019",T18PTQ_20190527T153621_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.14072,Ninguna,"8° 30' 47,551"" N","77° 19' 30,507"" W",ACANDÍ
2,7459,Polygon,"2,020",T18PTQ_20200112T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.14125,Ninguna,"8° 22' 30,966"" N","77° 13' 49,720"" W",ACANDÍ
3,9719,Polygon,"2,021",T18PTQ_20201202T153619_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.158066,Ninguna,"8° 36' 11.256"" N","77° 20' 50.757"" W",ACANDÍ
4,7464,Polygon,"2,020",T18PTQ_20200201T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.184549,Ninguna,"8° 22' 22,008"" N","77° 14' 3,921"" W",ACANDÍ


In [3]:
# Tipos de datos de cada columna
df.dtypes

ID                 int64
TIPO GEOMETRIA    object
AÑO               object
IMAGEN            object
CAUSA             object
AREA_Ha           object
OBSERVACION       object
LATITUD           object
LONGITUD          object
MUNICIPIO         object
dtype: object

In [4]:
# Resumen más completo
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7937 entries, 0 to 7936
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              7937 non-null   int64 
 1   TIPO GEOMETRIA  7937 non-null   object
 2   AÑO             7937 non-null   object
 3   IMAGEN          5901 non-null   object
 4   CAUSA           7718 non-null   object
 5   AREA_Ha         7937 non-null   object
 6   OBSERVACION     7937 non-null   object
 7   LATITUD         7930 non-null   object
 8   LONGITUD        7930 non-null   object
 9   MUNICIPIO       7937 non-null   object
dtypes: int64(1), object(9)
memory usage: 620.2+ KB


In [5]:
# Contar columnas categóricas - Normalmente son tipo Object o Category
cat_cols = df.select_dtypes(include=['object', 'category']).columns
num_cat = len(cat_cols)
print("Columnas categóricas:", num_cat)
print("Nombre:", list(cat_cols))

Columnas categóricas: 9
Nombre: ['TIPO GEOMETRIA', 'AÑO', 'IMAGEN', 'CAUSA', 'AREA_Ha', 'OBSERVACION', 'LATITUD', 'LONGITUD', 'MUNICIPIO']


In [6]:
# Contar columnas númericas int64 o float64
cont_cols = df.select_dtypes(include=["int64", "float64"]).columns
num_cont = len(cont_cols)
print("Columnas númericas:", num_cont)
print("Nombre:", list(cont_cols))

Columnas númericas: 1
Nombre: ['ID']


In [7]:
# Total de valores nulos por columna
print(df.isnull().sum())

ID                   0
TIPO GEOMETRIA       0
AÑO                  0
IMAGEN            2036
CAUSA              219
AREA_Ha              0
OBSERVACION          0
LATITUD              7
LONGITUD             7
MUNICIPIO            0
dtype: int64


In [8]:
# Verificar tipo de área de AREA_Ha
print("Tipo de dato original de AREA_Ha:", df["AREA_Ha"].dtype)

Tipo de dato original de AREA_Ha: object


In [9]:
# Convertir a númerico
df["AREA_Ha"] = pd.to_numeric(df["AREA_Ha"],errors="coerce")
print("Tipo de dato despues de convertir AREA_Ha:", df["AREA_Ha"].dtype)

Tipo de dato despues de convertir AREA_Ha: float64


In [10]:
# Tipos de datos de cada columna
df.dtypes

ID                  int64
TIPO GEOMETRIA     object
AÑO                object
IMAGEN             object
CAUSA              object
AREA_Ha           float64
OBSERVACION        object
LATITUD            object
LONGITUD           object
MUNICIPIO          object
dtype: object

In [11]:
# Revisar si hubo valores no convertidos
n_no_numericos = df["AREA_Ha"].isna().sum()
print(f"Valores no númericos convertidos a NaN: {n_no_numericos}")

Valores no númericos convertidos a NaN: 9
